# Energy-Magnitude notebook

Ampitude measurements, such as RSAM and Reduced Displacement, do not effectively capture the size of an event because some events of equal amplitude may be of very different durations. So it is useful to estimate the energy of the event. Furthermore, because event energy can range of several orders of magnitude, it is useful to define an energy magnitude.

We compute seismic source energy  (in J) via the equation:



(Boatwright, 198?, Johnson and Aster, 2003). 

Since this includes a correction for geometrical spreading, we will call it Reduced Energy ($E_{R}$).

The quantity ---- is similar to RSEM (Real-time Seismic Energy Measurement) but computed on a velocity seismogram rather than a raw seismogram, so we call it VSEM.

And then following Hanks and Kanamori (1???), we define Energy-Magnitude ($M_{E}$) as:

We will now compute $E_{S}$ and $M_{E}$ for the same events we examined in the Reduced Displacement tutorial.

References:

- Boatwright, ...
- Johnson & Aster, ...
- VSEM?
- Hanks and Kanamori ...


In [ ]:
import os
import sys
import obspy
sys.path.append('../lib')
from SAM import VSEM 

In [ ]:
# Load raw seismic data - and set units accordingly
mseedfile = os.path.join('..','..','data','continuous','SDS','2009','AV','REF','EHZ.D', 'AV.REF..EHZ.D.2009.081' )
st = obspy.read(mseedfile, 'MSEED')
for tr in st:
    tr.stats['units'] = 'Counts'
st.plot();

# load instrument response, remove instrument response from seismogram
stationxml = '../../data/responses/AV.xml'
inv = obspy.read_inventory(stationxml)
#st.detrend('linear')
#st.taper(0.05, type='hann')

# remove instrument response - and set units accordingly IMPORTANT!
pre_filt = [0.25, 0.5, 25, 50]
st.remove_response(inventory=inv, pre_filt=pre_filt, output="VEL", plot=True) 

In [ ]:
for tr in st:
    tr.stats['units'] = 'm2/s'
st.plot();

#################################################
# So far we just have a displacement seismogram
#################################################

# Generate a VSEM object - units must be 'm' else will not work
vsemObj = VSEM(stream=st, sampling_interval=600)
print(vsemObj)

# Define a source location for Redoubt volcano. Station distances to this lat/lon are used to "reduce" the displacement to 1 km distance.
source = {'lat':60.4845, 'lon':-152.7392}
print(source)

# Generate a Reduced Energy object, assuming body waves (surfaceWaves=False) and no inelastic attenuation (Q=None)
ERobj = vsemObj.compute_reduced_energy(inv, source, Q=None)
ERobj.plot(metrics='energy')

# Compute Energy Magnitude
ERobj.sum_energy()

## 3. Comparing the Reduced Energy and Energy Magnitude of several eruptions

1-minute VSEM data have been pre-computed for several eruptions. We will now load these, compute Reduced Energy, and compare them. 

As before, we will capture the highest reduced displacement into a results DataFrame, resultsDF, and we will compute Reduced Energy in two frequency bands. The default is 0.5-18.0 Hz. The second is a VLP passband, from 0.02 to 0.2 Hz. We write a function, compute_reduced_energies, so we don't have to keep repeating the same code block:

In [ ]:
import os
import sys
import glob
import pandas as pd
import obspy
sys.path.append('../lib')
import setup_paths
paths = setup_paths.paths
from SAM import VSEM

resultsDF = pd.DataFrame(columns=['Event', 'sum(ER)', 'ME', 'sum(ER_VLP)', 'ME_VLP'])

def compute_reduced_energies(eventname, vsemObj, source, stationxml, resultsDF):
    vsemObj = vsemObj.select(component='Z')
    inv = obspy.read_inventory(stationxml)
    
    # reduced energy
    ERobj = vsemObj.compute_reduced_energy(inv, source)
    ERobj.plot(metrics='energy')
    sumE, ME = ERobj.sum_energy(metric='energy')
    sumE_VLP, ME_VLP = ERobj.sum_energy(metric='VLP')
    
    # add to results
    resultsDF.loc[len(resultsDF.index)] = [eventname, sumE, ME, sumE_VLP, ME_VLP]
    display(resultsDF)

### 3.1 Event 1: Boxing Day Collapse 1997, Montserrat

29 months into the eruption of the Soufriere Hills Volcano, Montserrat, part of the crater wall the lava dome was growing within (and overtopping) suddenly collapsed in a landslide, causing a sideways explosion of the lava dome which effectively removed all traces of villages in the southwest quadrant of Montserrat (the villages had been evacuated more than a year earlier). At this time, there were only 2 stations operational, due to months of pyroclastic flows which had destroyed the capital, Plymouth, the airport, numerous other villages, and several seismic stations, and it was much too dangerous to replace them.

In [ ]:
# Event name
eventname1 = 'Boxing Day Collapse 1997'
source1 = {'lat':16.71111, 'lon':-62.17722}
stationxml1 = '../../data/responses/MV.xml'
startt1 = obspy.UTCDateTime(1997,12,26,6,30,0)
endt1 = obspy.UTCDateTime(1997,12,26,8,0,0)
vsemObj1 = VSEM.read(startt1, endt1, SAM_DIR=paths['SAM_DIR'], sampling_interval=60, ext='pickle', trace_ids=['MV.MBWH..SHZ', 'MV.MBLG..SHZ'])
compute_reduced_energies(eventname1, vsemObj1, source1, stationxml1, resultsDF)

In this example, for the regular passband, we get very similar values of reduced displacement whether we assume body waves or surface waves - both around 217 ${cm}^2$. In the VLP passband, the signal is 10 times smaller, which isn't surprising as the only stations available had short-period seismometers.

### 3.2 July 12th 2003 Dome collapse, Montserrat
On July 12th, 2003, over 200 million ${m}^3$ of the lava dome collapse in a series of explosions and pyroclastic flows, over a few hours. Let's compare the Reduced Displacement of this event to the Boxing Day 1997 collapse we just examined. Since this collapse happened mostly down the Tar River Valley, we will pick a source location there, rather than one centred on the lava dome.

In [ ]:
eventname2 = '2003/07/12 collapse, Montserrat'
stationxml2 = '../../data/responses/MV.xml'
source2 = {'lat':16.7164, 'lon':-62.1654}  # Tar River
startt2 = obspy.UTCDateTime(2003,7,12,23,0,0)
endt2 = obspy.UTCDateTime(2003,7,13,5,0,0)
vsemObj2 = VSEM.read(startt2, endt2, SAM_DIR=paths['SAM_DIR'], sampling_interval=60, ext='pickle', \
                    trace_ids=['MV.MBGH..BHZ', 'MV.MBLG..SHZ', 'MV.MBRY..BHZ', 'MV.MBSS..SHZ', 'MV.MBWH..SHZ'])
compute_reduced_energies(eventname2, vsemObj2, source2, stationxml2, resultsDF)

### 3.3 Phreatic eruption, Whakaari, Dec 9th, 2019

Whakaari - otherwise known as White Island - is the subaerial tip of a 1600 m high volcano that rises from the seafloor. Tourists arriving by boat and helicopter take a short hike into an open volcanic crater, past fumaroles, to an overlook of the crater lake. Phreatic eruptions are particularly difficult to forecast, as unlike magmatic eruptions, there are no seismic precursors (or indeed any other precuroses) that have been identified. Sadly, this particulary eruption led to the deaths of 22 tourists and guides. Let's compute the Reduced Displacement of this event.


In [ ]:
eventname3 = '2019/12/09 Phreatic explosion, Whakaari'
stationxml3 = '../../data/responses/NZ.xml'
source3 = {'lat':-37.5217, 'lon':177.185}
startt3 = obspy.UTCDateTime(2019,12,9,1,0,0)
endt3 = obspy.UTCDateTime(2019,12,9,1,30,0)
vsemObj3 = VSEM.read(startt3, endt3, SAM_DIR=paths['SAM_DIR'], sampling_interval=60, ext='pickle')
compute_reduced_energies(eventname3, vsemObj3, source3, stationxml3, resultsDF)

So this was a much smaller event - a Reduced Displacement of ~20 ${cm}^2$. It was also short in duration - just a few minutes long. But tourist groups inside the crater stood no chance.

### 3.4 Sub-Plinian eruption of Redoubt volcano, March 23, 2009

Redoubt volcano in Alaska had been in a state of unrest for 2-3 months, with deep-long-period earthquakes, deformation, swarms, and tremor, before explosively erupting at least 18 times between March 23rd and March 28th, 2009. We look only at March 23rd. There were actually 5 major explosive events on this day, which began at 6:38am (18,000 ft), 7:02am (44,000 ft), 08:14am (43,000 ft), 9:39am (43,000 ft), and 12:31pm (49,000 ft). 

The seismic data come from an analog telemetry system, and most channels are heavily contaminated by large interference spikes, and if we don't remove these, we'll end up with Reduced Displacement of ~10^7 ${cm}^2$ !! So we'll add an extra step to make sure the seismograms look good, before proceeding. It turns out there are only two stations free of this noise, RDN and REF, although these also become contaminated from around 3pm onwards.

In [ ]:
from obspy.clients.filesystem.sds import Client as sdsclient
mySDSclient = sdsclient(paths['SDS_DISP_DIR'])
startt4 = obspy.UTCDateTime(2009,3,23,6,0,0)
endt4 = obspy.UTCDateTime(2009,3,23,15,0,0)
st1 = mySDSclient.get_waveforms("AV", "RDN", "", "EHZ", startt4, endt4)
st2 = mySDSclient.get_waveforms("AV", "REF", "", "EHZ", startt4, endt4)
st = st1 + st2
for tr in st:
    tr.stats.units='m2/s'
st.plot(equal_scale=False);

The seismograms look clean, so now we'll compute the DSAM data and apply a clip level of 0.01 mm to the displacement seismogram. This might take 1-2 minutes because it is computing DSAM data for 24 hours of multi-channel data.

In [ ]:
vsemObj4 = VSEM(stream=st, sampling_interval=60)

eventname4 = '2009/03/23 Redoubt Sub-Plinian eruptions'
stationxml4 = '../../data/responses/AV.xml'
source4 = {'lat':60.4845, 'lon':-152.7392}
compute_reduced_energies(eventname4, vsemObj4, source4, stationxml4, resultsDF)

### 3.5 Hunga Tonga eruption, Jan 15th, 2022

Our final example is Hunga Tonga, which on January 15th, 2022, exploded spectacularly, destroying most of the island, generating a tsunami, and seismic, pressure and gravity waves that propagated around the globe several times. 

In [ ]:
from obspy.clients.filesystem.sds import Client as sdsclient
eventname5 = 'Hunga Tonga, 2022/01/15'
mySDSclient = sdsclient(paths['SDS_DISP_DIR'])
startt5 = obspy.UTCDateTime(2022,1,15,3,30,0)
endt5 = obspy.UTCDateTime(2022,1,15,10,0,0)
st = mySDSclient.get_waveforms("II", "MSVF", "10", "BHZ", startt5, endt5)
for tr in st:
    tr.stats.units='m2/s'
st.plot(equal_scale=False);

In [ ]:
source5 = {'lat':-20.57, 'lon':-175.38}
stationxml5 = '../../data/responses/II.xml'
vsemObj5 = VSEM(stream=st, sampling_interval=60)
compute_reduced_energies(eventname5, vsemObj5, source5, stationxml5, resultsDF)